In [1]:
import pandas as pd
import os
from datetime import datetime
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.inspection import plot_partial_dependence
from sklearn.inspection import partial_dependence 
from sklearn.metrics import classification_report


%matplotlib inline

In [115]:
nm2 = pd.read_pickle("data/all_calls17_uof_df.pkl")

# need to:
# make a y and make categories numeric(?)
# drop CAD and Event and Force
# make dummie columns
# cahnge month and day to integers


In [3]:
nm.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1459754 entries, 0 to 3986667
Data columns (total 14 columns):
CAD_Event_Number               1459754 non-null int64
Event_Clearance_Description    1459754 non-null object
Force_Level                    1459754 non-null object
Precinct                       1459754 non-null object
Sector                         1449873 non-null object
Beat                           1459754 non-null object
Subject_Race                   1459754 non-null object
Subject_Gender                 1459754 non-null object
Call_Type                      1459754 non-null object
Initial_Call_Type              1459754 non-null object
month                          1459754 non-null object
day                            1459754 non-null object
day_of_week                    1459754 non-null int64
hour                           1459754 non-null int64
dtypes: int64(3), object(11)
memory usage: 167.1+ MB


In [4]:
y = nm.Force_Level
y.replace({"Level 1 - Use of Force":1,"Level 2 - Use of Force":2,"Level 3 - OIS":4, "Level 3 - Use of Force": 3 }, inplace=True)


In [5]:
# make a copy to alter for the model in case I want to look back at this
nm2 = nm.copy(deep=True)

In [6]:
# drop columns and make time ints
nm.drop(["Force_Level", "CAD_Event_Number", "Event_Clearance_Description"], axis=1, inplace=True)
nm["month"] = nm["month"].astype(int)
nm["day"] = nm["day"].astype(int)

In [7]:
# better categories in Initial Call Type

init = nm["Initial_Call_Type"]
to_drop = init[init.str.startswith("-ASSIGNED DUTY")]


s = nm["Initial_Call_Type"].str.replace(" - .*", "", regex=True)
nm["Initial_Call_Type"] = s

In [ ]:
#drop = ["ASSIST PUBLIC", ]  -DOWN TIME - OUT OF SERVICE  

In [324]:
# cool EDA

call_types = nm2.groupby(["Call_Type"]).Initial_Call_Type.value_counts()

df1 = nm2[nm2['cat_ICT'].str.contains("LITTERING") ]

vc = nm2.cat_ICT.value_counts()
vc[70:]
#df1.cat_ICT.value_counts()

Animal - Dangerous / Bite                        847
Arson                                            838
ILLEGAL DUMPING                                  816
BURN - RECKLESS BURNING                          808
Bias                                             795
LICENSE INSPECTIONS (CHECK FOR)                  731
HARBOR - WATER EMERGENCIES                       721
Sleeper Aboard Public Transit                    671
Explosion                                        644
INFORMATIONAL BROADCASTS                         592
Harassment                                       571
TEST CALL ONLY                                   521
ABANDONED VEHICLE                                395
-OUT AT RANGE                                    328
-OUT OF CAR/NO REASON GIVEN                      315
Bomb Threat                                      243
WARRANT - SEARCH. EXECUTED, SECURED              209
POWER OUT - POLES AND TRANSFORMERS               189
Eluding Police                                

In [325]:
# function for later to categorize ICT column

def initial_call_type_cats(value):    
    if "MVC" in value:
        return "Motor Vehicle Collision"
    elif value.startswith("FRAUD"):
        return "Fraud"
    elif "SUSPICIOUS STOP" in value:
        return "Police Initiated"
    elif "SUSPICIOUS" in value:
        return "Suspicious person, vehicle, incident"
    elif "TRAFFIC STOP" in value:
        return "Police Initiated"
    elif "ROAD RAGE" in value:
        return "Road Rage"
    elif "MOVING VIOLATION" in value:
        return "Moving Violation"
    elif "TRAF" in value:
        return "Moving Violation"
    elif "BLOCKING ROADWAY" in value:
        return "Blocking Roadway"
    elif "TRAFFIC - ASSIST" in value:
        return "Assist Motorist"
    elif "DISTURBANCE" in value:
        return "Disturbance"
    elif "NOISE" in value:
        return "Disturbance"
    elif "OFF DUTY EMPLOYMENT" in value:
        return "Police Initiated"
    elif "NUISANCE" in value:
        return "Nuisance"
    elif "NARCOTICS" in value:
        return "Narcotics"
    elif "SEX IN PUBLIC" in value:
        return "Sex in Public"
    elif "VIOLATION OF COURT ORDER (NON DV)" in value:
        return "Violation of Court Order (Not Domestic Violence)"
    elif "NOT DV" in value:
        return "Alarm"
    elif "SHOTS" in value:
        return "Shots heard"
    elif "DV" in value:
        return "Domestic Violence"
    elif "ASSAULTS" in value:
        return "Assault"
    elif "WEAPN" in value:
        return "Deadly Weapon - No Assault"
    elif "ASLT" in value:
        return "Assault"
    elif "FIGHT" in value:
        if "WITH WEAPONS" in value:
            return "Fight - with weapons"
        else:
            return "Fight - no weapons"
    elif "THEFT" in value:
        return "Robbery"
    elif "PREMISE CHECK" in value:
        return "Police Initiated"
    elif "DIRECTED PATROL ACTIVITY" in value:
        return "Directed Patrol Activity"
    elif "UNUSUAL OCCURRENCES - EARTHQUAKE, RIOT, ETC)" in value:
        return "Disturbance"
    elif "PARKING" in value:
        return "Parking Violation"
    elif "ALARM"in value:
        return "Alarm"
    elif "BURG" in value:
        return "Robbery"
    elif "WATER FLOODS" in value:
        return "Nuisance"
    elif "TREES DOWN" in value:
        return "Nuisance"
    elif "HAZ" in value:
        return "Hazard - immediate threat"
    elif "TRESPASS" in value:
        return "Trespass"
    elif "FOLLOW" in value:
        return "Follow up"
    elif "WELFARE" in value:
        return "Welfare Check"
    elif "SERVICE" in value:
        return "Assisting other agencies"
    elif "SFD" in value:
        return "Assisting other agencies"
    elif "SUICIDE" in value:
        return "Suicidal person or attempts"
    elif "LIQUOR" in value:
        return "Liquor Violations"
    elif "AUTO" in value:
        return "Auto Recovery"
    elif "DAMAGE" in value:
        return "Damaged Property"
    elif "DAMG" in value:
        return "Damaged Property"
    elif "PROPERTY" in value:
        return "Property lost or found"
    elif "FOLLOW UP" in value:
        return "Follow up"
    elif "PERSON IN BEHAVIORAL/EMOTIONAL CRISIS" in value:
        return "Behavioral / Emotional Crisis"
    elif "DOWN - CHECK" in value:
        return "Check for Person Down"
    elif "SICK" in value:
        return "Sick Person"
    elif "--PERSON" in value:
        return "Missing Person"
    elif "INJURED" in value:
        return "Injuries from Industrial Accidents"
    elif "PICKUP" in value:
        return "Pickup Person"
    elif "BOMB" in value:
        return "Bomb Threat"
    elif "THREAT" in value:
        return "Threats"
    elif "RAPE" in value:
        return "Rape"
    elif "ROBBERY" in value:
        return "Robbery"
    elif "KIDNAPPING" in value:
        return "Missing Person"
    elif "MISSING" in value:
        return "Missing Person"
    elif "PROWLER" in value:
        return "Prowler"
    elif "PANHANDLING" in value:
        return "Aggressive Panhandling"
    elif "RUNAWAY" in value:
        return "Missing Person"
    elif "ASSIGNED DUTY" in value:
        return "Assigned Duty"
    elif "KIDNAPPNG" in value:
        return "Missing Person"
    elif "LEWD" in value:
        return "Lewd Conduct"
    elif "OVERDOSE" in value:
        return "Overdose - Drug Related Casualty"
    elif "PREDICTIVE POLICING" in value:
        return "Predictive Policing"
    elif "DETOX" in value:
        return "Request for Detox"
    elif "REQUEST" in value:
        return "Request to Watch"
    elif "CHILD" in value:
        return "Child Abuse"
    elif "PURSE" in value:
        return "Robbery"
    elif "PEACE" in value:
        return "Standby to Ensure Peace"
    elif "DEAD" in value:
        return "Casualty"
    elif "GAMBLING" in value:
        return "Gambling"
    elif "ANIMAL" in value:
        return "Animal - Dangerous / Bite"
    elif "ESCAPE" in value:
        return "Escaped Prisoner"
    elif "PARKS" in value:
        return "Parks Violations"
    elif "PROSTITUTION" in value:
        return "Prostitution"
    elif "Pornography" in value:
        return "Lewd Conduct"
    elif "DUI" in value:
        return "DUI - Driving While Under the Influence"
    elif "SEX OFFENDER" in value:
        return "Sex Offender - Failure to Register"
    elif "SEX" in value:
        return "Bias"
    elif "SLEEP" in value:
        return "Sleeper Aboard Public Transit"
    elif "HARASSMENT" in value:
        return "Harassment"
    elif "EXPLOSION" in value:
        return "Explosion"
    elif "ARSON" in value:
        return "Arson"
    elif "PREPLANNED" in value:
        return "Assigned Duty"
    elif "ASSIGNMENT" in value:
        return "Assigned Duty"
    elif "SHOPLIFT" in value:
        return "Robbery"
    elif "ELUDING" in value:
        return "Eluding Police"
    elif "LITTERING" in value:
        return "Illegal Dumping"
    elif "DUMPING" in value:
        return "Illegal Dumping"
    elif "BURN" in value:
        return "Arson"
    
    
    
    
    
    
    
    
    
    
    
    
    
    else:
        return "-"
    
    
    

In [326]:
# calling above function to make list to make column
new_col2 = []
for item in new_col:
   # value = row.Initial_Call_Type
    new_col2.append(initial_call_type_cats(item))

In [327]:
nm2["cat_ICT"] = new_col

In [8]:
# get dummies for string valued columns
nm = pd.get_dummies(data = nm, 
                     columns=["Precinct", "Sector", "Beat", "Subject_Gender", "Subject_Race", "Call_Type", 
                               "Initial_Call_Type"], 
                     prefix=["P",  "S", "B", "SG", "SR","CT", "ICT"])

In [26]:
# dropping the "-" Precinct and Sector columns - they aren't useful for interpretation

nm.drop(["S_-", "P_-"], axis=1, inplace=True)

In [33]:
# dropping column

nm.drop(['SG_-', 'SR_-'], axis=1, inplace=True)

In [36]:
# get column names to identify features in feature_importance later

cols = nm.columns

In [19]:
# two minutes~

X_scaled = StandardScaler().fit_transform(nm)

In [73]:
# will need to change nm2 to X_scaled if we use this later - X_scaled hung and never finished (~10minutes)

X_train, X_test, y_train, y_test = train_test_split(nm, y, test_size=0.2, random_state=659)

In [74]:
# create RFC model - change all but random_state after we time this one

clf = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=659, oob_score=True)



In [75]:
# fit the model

clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=659, verbose=0,
                       warm_start=False)

In [76]:
# check the feature importances and col_name list match first

len(clf.feature_importances_), len(cols)

(347, 347)

In [77]:
# most important features in predictions of this model

indices = np.argsort(clf.feature_importances_)
big = indices[-1:-20:-1]

cols[big]

Index(['SG_Male', 'SR_Black or African American', 'SR_White', 'P_W',
       'SG_Female', 'P_N', 'P_E', 'S_K', 'S_M', 'S_N', 'SR_Not Specified',
       'P_S', 'S_E', 'S_R', 'S_S', 'S_G', 'S_B', 'P_SW', 'S_U'],
      dtype='object')

In [78]:
# least important features
little = indices[:10]
cols[little]


Index(['B_TAC3', 'B_T', 'B_TAC2', 'B_TAC9', 'B_TQ', 'B_TRF',
       'ICT_KNOWN KIDNAPPNG', 'B_US', 'B_W', 'B_W3'],
      dtype='object')

In [79]:
clf.oob_score_ # accuracy of training set

0.9953065713994569

In [80]:
# predicted classes

yhat = clf.predict(X_test)

In [81]:
sum(yhat == y_test) / len(yhat) # accuracy of testing set

0.9948313244345798

In [82]:
# sklearn.metrics.classification_report(y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False)
print(classification_report(y_test, yhat))


/Users/sarahburgart/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    290181
           1       0.74      0.20      0.31      1320
           2       0.00      0.00      0.00       414
           3       0.00      0.00      0.00        26
           4       0.00      0.00      0.00        10

    accuracy                           0.99    291951
   macro avg       0.35      0.24      0.26    291951
weighted avg       0.99      0.99      0.99    291951



In [83]:
# predict probabilities - what do you want threshold to be?

pp = clf.predict_proba(X_test)

In [106]:
clf.classes_

array([0, 1, 2, 3, 4])

In [107]:
# some rows' probabilities of being in each class:

for p in pp[:10]:
    print(f"Pr of no force is: {p[0]}, Pr of Force 1 is: {p[1]}, Pr of Force 2: {p[2]}, Pr of Force 3: {p[3]}, & Pr of Officer Involved Shooting: {p[4]}")
          
          

Pr of no force is: 0.9975696041175851, Pr of Force 1 is: 0.001786008381623894, Pr of Force 2: 0.0005890931487029798, Pr of Force 3: 3.458282343925955e-05, & Pr of Officer Involved Shooting: 2.0711528648684556e-05
Pr of no force is: 0.9975436430474433, Pr of Force 1 is: 0.001804144082206821, Pr of Force 2: 0.0005963763448513798, Pr of Force 3: 3.495330860313598e-05, & Pr of Officer Involved Shooting: 2.0883216895359e-05
Pr of no force is: 0.997591777167731, Pr of Force 1 is: 0.001765743712719444, Pr of Force 2: 0.0005879582209436118, Pr of Force 3: 3.446763998622903e-05, & Pr of Officer Involved Shooting: 2.0053258619722154e-05
Pr of no force is: 0.9973018147227163, Pr of Force 1 is: 0.001966689066885495, Pr of Force 2: 0.0006691257202245938, Pr of Force 3: 3.965555021180126e-05, & Pr of Officer Involved Shooting: 2.2714939961815766e-05
Pr of no force is: 0.997185974091859, Pr of Force 1 is: 0.0020558252080745167, Pr of Force 2: 0.0006951011948231337, Pr of Force 3: 3.9539936729357115e-

In [85]:
len(y_test)

291951

In [86]:
type(y_test)

pandas.core.series.Series

In [108]:
y_test[y_test == 4]

50706     4
220355    4
156571    4
60263     4
6215      4
156570    4
102803    4
50750     4
220432    4
50774     4
Name: Force_Level, dtype: int64

In [109]:
type(yhat)

numpy.ndarray

In [111]:
len(yhat[yhat == 4])

0

In [90]:
yhat[50706]

0

In [118]:
# visualization - you can plot the abundance/frequency of each selected feature within each group as a bar plot. 
# I assume visually the top feature will be more abundant in one group comparing with the other groups.

four = y_test[y_test == 4]
four, four.index

(50706     4
 220355    4
 156571    4
 60263     4
 6215      4
 156570    4
 102803    4
 50750     4
 220432    4
 50774     4
 Name: Force_Level, dtype: int64,
 Index([50706, 220355, 156571, 60263, 6215, 156570, 102803, 50750, 220432,
        50774],
       dtype='object'))

In [136]:
fourX = X_test.iloc[[50706, 220355, 156571, 60263, 6215, 156570, 102803, 50750, 220432,
        50774],:]
fourX


,month,day,day_of_week,hour,P_E,P_EAST,P_N,P_NORTH,P_S,P_SOUTH,...,ICT_TRU,ICT_UNKNOWN,ICT_UNUSUAL OCCURRENCES,ICT_VICE,ICT_WARRANT,ICT_WARRANT PICKUP,"ICT_WATER FLOODS (BROKEN MAINS/HYDRANTS, NO HAZ)",ICT_WEAPN,"ICT_WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)","ICT_WIRES DOWN (PHONE, ELECTRICAL,ETC.)"
2532371,1,21,3,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1847045,4,14,0,9,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2560580,2,16,1,12,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2805351,9,17,5,23,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1984306,8,18,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2743392,7,25,0,15,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3932163,5,12,6,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3193740,8,23,2,20,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2677576,5,30,0,13,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3875946,3,26,1,13,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [145]:
fournm = nm2.iloc[50706,:]
fournm

four_index = [50706, 220355, 156571, 60263, 6215, 156570, 102803, 50750, 220432,
        50774]

nm2[nm2.index.isin(four_index)]

,CAD_Event_Number,Event_Clearance_Description,Force_Level,Precinct,Sector,Beat,Subject_Race,Subject_Gender,Call_Type,Initial_Call_Type,month,day,day_of_week,hour
6215,2014000431101,ASSISTANCE RENDERED,Level 3 - OIS,S,S,S2,Black or African American,Male,911,SUICIDE - IP/JO SUICIDAL PERSON AND ATTEMPTS,12,31,2,21
50706,2014000431106,UNABLE TO LOCATE INCIDENT OR COMPLAINANT,Level 3 - OIS,S,S,S2,Black or African American,Male,ONVIEW,SHOTS - IP/JO - INCLUDES HEARD/NO ASSAULT,12,31,2,21
50750,2014000431101,ASSISTANCE RENDERED,Level 3 - OIS,S,S,S2,Black or African American,Male,911,SUICIDE - IP/JO SUICIDAL PERSON AND ATTEMPTS,12,31,2,21
50774,2014000431106,UNABLE TO LOCATE INCIDENT OR COMPLAINANT,Level 3 - OIS,S,S,S2,Black or African American,Male,ONVIEW,SHOTS - IP/JO - INCLUDES HEARD/NO ASSAULT,12,31,2,21
60263,2014000431101,ASSISTANCE RENDERED,Level 3 - OIS,S,S,S2,Black or African American,Female,911,SUICIDE - IP/JO SUICIDAL PERSON AND ATTEMPTS,12,31,2,21
102803,2015000340290,PHYSICAL ARREST MADE,Level 3 - OIS,W,K,K1,Not Specified,Male,"TELEPHONE OTHER, NOT 911",THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING),09,29,1,2
156570,2019000098794,ASSISTANCE RENDERED,Level 3 - OIS,-,-,E1,Asian,Male,911,"SUSPICIOUS PERSON, VEHICLE OR INCIDENT",03,19,1,21
156571,2019000098792,REPORT WRITTEN (NO ARREST),Level 3 - OIS,-,-,Q1,Asian,Male,911,THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING),03,19,1,21
220355,2017000456733,ASSISTANCE RENDERED,Level 3 - OIS,N,U,U3,White,Male,911,"SUSPICIOUS PERSON, VEHICLE OR INCIDENT",12,11,0,22
220432,2017000456733,ASSISTANCE RENDERED,Level 3 - OIS,N,U,U3,White,Male,911,"SUSPICIOUS PERSON, VEHICLE OR INCIDENT",12,11,0,22


In [142]:
fournm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 1817309 to 1817387
Data columns (total 14 columns):
CAD_Event_Number               10 non-null int64
Event_Clearance_Description    10 non-null object
Force_Level                    10 non-null object
Precinct                       10 non-null object
Sector                         10 non-null object
Beat                           10 non-null object
Subject_Race                   10 non-null object
Subject_Gender                 10 non-null object
Call_Type                      10 non-null object
Initial_Call_Type              10 non-null object
month                          10 non-null object
day                            10 non-null object
day_of_week                    10 non-null int64
hour                           10 non-null int64
dtypes: int64(3), object(11)
memory usage: 1.2+ KB
